<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 4.6MB/s 
     |████████████████████████████████| 890kB 24.2MB/s 
     |████████████████████████████████| 3.0MB 42.2MB/s 
     |████████████████████████████████| 1.1MB 59.0MB/s 
     |████████████████████████████████| 36.2MB 83kB/s 
     |████████████████████████████████| 1.5MB 56.0MB/s 


In [2]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
import re
parsivar_tokenizer = Tokenizer()
parsivar_normalizer = Normalizer()

In [14]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()
print()

# **Tokenize**

### Example

In [3]:
tweet_example = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

tweet_example2 = """
 محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست

"""

tweet_example3 = """
سلام
خواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. 
چند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.

"""

In [23]:
input_sentence = tokenizer.tokenize(tweet_example3)
for token in input_sentence:
  print(token)

سلام
خواهر
من
فکر
کنم
دو
سالی
سرفه
امانش
را
بریده
بود
.
خیلی
کارها
کرد
برای
درمانش
اما
درست
نشد
که
نشد
.
کرونا
هم
که
شروع
شد
،
خیلی
نگرانش
بودیم
.
چند
روز
پیش
با
چند
تا
نیش
زنبور
عسل
به
اصطلاح
زنبور
درمانی
کرد
و
با
کمال
تعجب
دیگه
سرفه
نمی
کنه
.


In [4]:
for token in parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_example3)):
  print(token)

سلام
خواهر
من
فکر
کنم
دو
سالی
سرفه
امانش
را
بریده‌بود
.
خیلی
کارها
کرد
برای
درمانش
اما
درست
نشد
که
نشد
.
کرونا
هم
که
شروع
شد
،
خیلی
نگرانش‌بودیم
.
چند
روز
پیش
با
چند
تا
نیش
زنبور
عسل
به
اصطلاح
زنبور
درمانی
کرد
و
با
کمال
تعجب
دیگه
سرفه
نمی‌کنه
.


### Tokenize All tweets

In [5]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

--2020-10-05 08:09:07--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2698436 (2.6M) [text/plain]
Saving to: ‘extracted_data.txt’

extracted_data.txt  100%[===================>]   2.57M  --.-KB/s    in 0.1s    

2020-10-05 08:09:07 (20.4 MB/s) - ‘extracted_data.txt’ saved [2698436/2698436]



In [6]:
! head -n 40 extracted_data.txt

خرداد ۱۳۹۹ «کیهان» میراث سناتور مصطفی مصباح‌زاده ۷۸ ساله شد 
کیهان لندن تنها رسانه‌ ایرانی در تبعید است که «از جنگ دوم جهانی تاکنون قدمت و یک مؤسسه‌ غصب ‌شده در تهران» دارد.



**************

چهارمین قهرمانی فلومیننسه در برزیل:  

**************

زندگی_سلام
 کوراسائو، بلیز، سیشل و نائورو!
‌
 خراسان_ورزشی ‌
‌دو بی انضباطی و دو واکنش متفاوت از فرهاد
 سرخ های پایتخت در اندیشه دبل 

**************

سال ۲۰۱۳ را سالی که در آن بایرن همه جام های ممکن را برد. 
و حالا ۲۰۲۰ تکرار کسب تمامی عناوین ممکن.
لیگ_قهرمانان
بوندس_لیگا
جام_حذفی_آلمان 
سوپرکاپ_اروپا
( سال ۲۰۱۳ ، قهرمان جام باشگاههای جهان) 

**************

 زین سرای بی کَسی ، کَس مجو
محنت عاشقی ز بی کَس  مجو 

طرب عاشقانه ایی روا دار کنون 
به کَس و بی کَس  کنون عشق مجو 

مجتبی_الوندی


**************


In [7]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'خرداد ۱۳۹۹ «کیهان» م'

In [8]:
tweets = tweets_str.split("\n\n**************\n\n")
tweets[0], len(tweets)

('خرداد ۱۳۹۹ «کیهان» میراث سناتور مصطفی مصباح\u200cزاده ۷۸ ساله شد \nکیهان لندن تنها رسانه\u200c ایرانی در تبعید است که «از جنگ دوم جهانی تاکنون قدمت و یک مؤسسه\u200c غصب \u200cشده در تهران» دارد.\n\n',
 9349)

In [9]:
len(tweets)

9349

In [10]:
len(set(tweets))

9349

In [11]:
# tweets_tokens = []
# for tweet in tweets:
#   tweet_tokens = tokenizer.tokenize(tweet)
#   tweets_tokens.append(tweet_tokens)

In [14]:
tweets_tokens = []
for tweet in tweets:
  tweet_tokens = re.sub(' +', ' ',
                        tweet.replace('\n', ' ').
                        replace('.', ' ').
                        replace('،', ' ').
                        replace('!', ' ').
                        replace('?', ' ').
                        replace(':', ' ').
                        replace('؟', ' ').
                        replace('_', ' ').
                        replace('|', ' ').
                        replace('/', ' ').
                        replace('+', ' ').
                        replace('[', ' ').
                        replace(']', ' '))
  tweet_tokens = parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_tokens))
  tweets_tokens.append(tweet_tokens)

In [16]:
tweets_tokens[0]

['خرداد',
 '1399',
 '«',
 'کیهان',
 '»',
 'میراث',
 'سناتور',
 'مصطفی',
 'مصباح\u200cزاده',
 '78',
 'ساله',
 'شد',
 'کیهان',
 'لندن',
 'تنها',
 'رسانه',
 'ایرانی',
 'در',
 'تبعید',
 'است',
 'که',
 '«',
 'از',
 'جنگ',
 'دوم',
 'جهانی',
 'تاکنون',
 'قدمت',
 'و',
 'یک',
 'موسسه',
 'غصب',
 'شده',
 'در',
 'تهران',
 '»',
 'دارد']

In [17]:
counter = 0
for i in range(1, 11):
  with open('persian-ner-twitter-data' + str(i) + '.txt', 'w') as file:
    for tweet_tokens in tweets_tokens[(i-1)*1000:max(len(tweets_tokens), i*1000)]:
      for token in tweet_tokens:
        if token != '':
          counter += 1
          file.write(token + '\tO\n')
      file.write('\n')
print(counter)

1389505


In [18]:
! head -n 50 persian-ner-twitter-data1.txt

خرداد	O
1399	O
«	O
کیهان	O
»	O
میراث	O
سناتور	O
مصطفی	O
مصباح‌زاده	O
78	O
ساله	O
شد	O
کیهان	O
لندن	O
تنها	O
رسانه	O
ایرانی	O
در	O
تبعید	O
است	O
که	O
«	O
از	O
جنگ	O
دوم	O
جهانی	O
تاکنون	O
قدمت	O
و	O
یک	O
موسسه	O
غصب	O
شده	O
در	O
تهران	O
»	O
دارد	O

چهارمین	O
قهرمانی	O
فلومیننسه	O
در	O
برزیل	O

زندگی	O
سلام	O
کوراسائو	O
بلیز	O
سیشل	O
و	O


In [19]:
! head -n 50 persian-ner-twitter-data5.txt

دوران	O
دفاع	O
مقدس	O
اوج	O
افتخارات	O
ملت	O
ایران	O
است	O
مقام	O
معظم	O
رهبری	O
هفته	O
دفاع	O
مقدس	O
غبارروبی	O
مزار	O
شهدای	O
روستای	O
ارتش	O
آباد	O
زادگاه	O
شهید	O
طیب	O
حاج	O
رضایی	O
پنجشنبه	O
99	O
7	O
3	O

توییت	O
لری	O
هی	O
برار	O
جو	O
تا	O
بیه	O
هی	O
روزگارو	O
هی	O
چنینه	O
جا	O
پاییز	O
و	O
جا	O
زمسّو	O
همیشه	O
بهار	O
نشینه	O
از	O


In [20]:
! head -n 50 persian-ner-twitter-data10.txt

خرسند	O

هند	O
در	O
حال	O
مذاکره	O
با	O
اروپا	O
برای	O
ادامه	O
واردات	O
نفت	O
از	O
ایران	O
علی‌رغم	O
تحریم‌ها	O
است	O
«	O
هندوستان‌تایمز	O
»	O
نوشت	O
طبق	O
گفته	O
منابع	O
رسمی	O
هند	O
جدا	O
از	O
تلاش	O
برای	O
مجاب	O
کردن	O
آمریکا	O
مبنی	O
بر	O
اینکه	O
تحریم‌های	O
این	O
کشور	O
علیه	O
ایران	O
پس	O
از	O
خروج	O
از	O
برجام	O
بر	O
اقتصاد	O
هند	O
اثرات	O


# **Model**

In [ ]:
from transformers import pipeline

In [ ]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet_example)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-organization',
  'index': 8,
  'score': 0.9995940327644348,
  'word': 'اتلتیکو'},
 {'entity': 'B-organization',
  'index': 13,
  'score': 0.9972712397575378,
  'word': 'بایرن'},
 {'entity': 'B-person',
  'index': 31,
  'score': 0.9915841817855835,
  'word': 'هاینکس'}]